In [ ]:
#單一使用訓練器結果:
#LogisticRegression = 0.8763796909492274 , GaussianNB = 0.8719646799116998 , SVC = 0.8763796909492274

#使用VotingClassifier 以LogisticRegression、GaussianNB、SVC 做訓練
#voting='hard' = 0.8763796909492274 , voting='soft' = 0.8763796909492274

#使用BaggingClassifier 以LogisticRegression、GaussianNB、SVC 做訓練
#GaussianNB = 0.8847099311701082 , SVC = 0.8856932153392331 , LogisticRegression = 0.8856932153392331

#使用AdaBoostClassifier 以GaussianNB、LogisticRegression做訓練
#GaussianNB = 0.3576158940397351 , LogisticRegression = 0.8763796909492274

#使用StackingClassifier 以LogisticRegression、GaussianNB、SVC 做訓練
# final_estimator = SVC 0.8763796909492274 , final_estimator = GaussianNB 0.8609271523178808 
# final_estimator = LogisticRegression 0.8785871964679912

#最好的結果是使用BaggingClassifier
#SVC = 0.8856932153392331 , LogisticRegression = 0.8856932153392331

In [132]:
import pandas as pd
data = pd.read_csv("bank/bank.csv",delimiter=';',header='infer')
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [108]:
final = data.drop(['job','marital','education','default','housing',
        'contact','day','month','duration','campaign','pdays','previous','poutcome'],axis=1)
final.head()

,age,balance,loan,y
0,30,1787,no,no
1,33,4789,yes,no
2,35,1350,no,no
3,30,1476,yes,no
4,59,0,no,no


In [109]:
final.isnull().sum() #查看資料是否有缺失 > 都沒有缺失

age        0
balance    0
loan       0
y          0
dtype: int64

將資料轉為有用資訊

In [110]:
final.y.replace(('yes','no'),(1,0),inplace=True)
final.loan.replace(('yes','no'),(1,0),inplace=True)
final.head()

,age,balance,loan,y
0,30,1787,0,0
1,33,4789,1,0
2,35,1350,0,0
3,30,1476,1,0
4,59,0,0,0


將Input、Output 定義成 X Y

In [111]:
X = final.drop(['y'],axis=1)
Y = final.drop(['age','balance','loan'],axis=1)
X.head()

,age,balance,loan
0,30,1787,0
1,33,4789,1
2,35,1350,0
3,30,1476,1
4,59,0,0


In [112]:
Y.head()

,y
0,0
1,0
2,0
3,0
4,0


將資料切分成訓練集80%、測試集20%

In [113]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)


In [114]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,Y_train)
lr.score(X_test,Y_test)

C:\Users\z3020\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8763796909492274

In [115]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train,Y_train)
gnb.score(X_test,Y_test)

C:\Users\z3020\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8719646799116998

In [116]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train,Y_train)
svc.score(X_test,Y_test)

C:\Users\z3020\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8763796909492274

In [117]:
from sklearn.ensemble import VotingClassifier
clf1 = LogisticRegression()
clf2 = GaussianNB()
clf3 = SVC()
eclf1 = VotingClassifier(estimators=[('lr',clf1),('gnb',clf2),('svc',clf3)],voting='hard') #hard分類結果0 1
eclf1 = eclf1.fit(X_train,Y_train)
eclf1.score(X_test,Y_test)

C:\Users\z3020\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\z3020\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8763796909492274

In [118]:
eclf2 = VotingClassifier(estimators=[('lr',clf1),('gnb',clf2),('svc',clf3)],voting='soft') #hard分類結果0 1
eclf2 = eclf1.fit(X_train,Y_train)
eclf2.score(X_test,Y_test)

C:\Users\z3020\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\z3020\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8763796909492274

In [119]:
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import GaussianNB
ebc1 = BaggingClassifier(base_estimator=GaussianNB(),n_estimators=100,max_features=0.8,bootstrap_features=True,max_samples=0.6)
ebc1.fit(X_train,Y_train)
ebc1.score(X_train,Y_train)

C:\Users\z3020\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\ensemble\_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8847099311701082

In [120]:
from sklearn.svm import SVC
ebc2 = BaggingClassifier(base_estimator=SVC(),n_estimators=100,max_features=0.8,bootstrap_features=True,max_samples=0.6)
ebc2.fit(X_train,Y_train)
ebc2.score(X_train,Y_train)

C:\Users\z3020\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\ensemble\_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8856932153392331

In [121]:
from sklearn.linear_model import LogisticRegression
ebc3 = BaggingClassifier(base_estimator=LogisticRegression(),n_estimators=100,max_features=0.8,bootstrap_features=True,max_samples=0.6)
ebc3.fit(X_train,Y_train)
ebc3.score(X_train,Y_train)

C:\Users\z3020\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\ensemble\_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8856932153392331

In [122]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
Ada1 = AdaBoostClassifier(base_estimator=GaussianNB(),n_estimators=100)
Ada1.fit(X_train,Y_train)
Ada1.score(X_test,Y_test)

C:\Users\z3020\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\z3020\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(


0.3576158940397351

In [123]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
Ada2 = AdaBoostClassifier(base_estimator=LogisticRegression(),n_estimators=100)
Ada2.fit(X_train,Y_train)
Ada2.score(X_test,Y_test)

C:\Users\z3020\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\z3020\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(


0.8763796909492274

In [124]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

esv1 = StackingClassifier(estimators=[('lr',LogisticRegression()),('gnb',GaussianNB()),('svc',SVC())],final_estimator=SVC())
esv1.fit(X_train,Y_train)
esv1.score(X_test,Y_test)

C:\Users\z3020\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\z3020\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8763796909492274

In [125]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

esv2 = StackingClassifier(estimators=[('lr',LogisticRegression()),('gnb',GaussianNB()),('svc',SVC())],final_estimator=GaussianNB())
esv2.fit(X_train,Y_train)
esv2.score(X_test,Y_test)

C:\Users\z3020\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\z3020\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8609271523178808

In [126]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

esv3 = StackingClassifier(estimators=[('lr',LogisticRegression()),('gnb',GaussianNB()),('svc',SVC())],final_estimator=LogisticRegression())
esv3.fit(X_train,Y_train)
esv3.score(X_test,Y_test)

C:\Users\z3020\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\z3020\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8785871964679912